In [ ]:
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra

# data visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import shap

# sklearn 
from sklearn.cluster import KMeans
from sklearn.preprocessing import PowerTransformer, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, silhouette_samples, accuracy_score, classification_report

from pyod.models.ecod import ECOD
from yellowbrick.cluster import KElbowVisualizer

import lightgbm as lgb
import prince

In [ ]:
# import os
# import re

# # Define a function to preprocess text
# def preprocess_text(text):
    
#     text = text.lower()
#     # Split the text into sentences
#     sentences = re.split(r'[.!?]', text)
    
#     for i,sentence in enumerate(sentences):
#         sentence = sentence.split("|")[-1]
#         sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
#         sentences[i] = sentence.split("|")[-1]
    
#     return sentences

# # Load the dataset from a folder
# dataset_folder = "documents/topic_model_dataset"
# sentences = []

# # Iterate over files in the dataset folder
# for filename in os.listdir(dataset_folder):
#     if filename.endswith(".txt"):
#         with open(os.path.join(dataset_folder, filename), "r", encoding="utf-8") as file:
#             text = file.read()
#             sentences.extend(preprocess_text(text))



# print(f'length of documents: {len(sentences)}')
# print(f"first sentences:\n", "\n".join(sentences[:5]))

sentences = ["This is a sentence about sports number " + str(i) for i in range(1, 26)] + \
                 ["This is a sentence about politics number " + str(i) for i in range(1, 26)] + \
                 ["This is a sentence about technology number " + str(i) for i in range(1, 26)] + \
                 ["This is a sentence about art number " + str(i) for i in range(1, 26)]



In [ ]:
from pipeline.embeddings.basic_embeddings import Embedding
import pandas as pd

model = Embedding().get_llm()

output = model.encode(sentences=sentences,
         show_progress_bar=True,
         normalize_embeddings=True)

df_embedding = pd.DataFrame(output)
df_embedding

In [ ]:
from pyod.models.ecod import ECOD

clf = ECOD()
clf.fit(df_embedding)


out = clf.predict(df_embedding) 
df_embedding["outliers"] = out

# df["outliers"] = out

df_embedding_no_out = df_embedding[df_embedding["outliers"] == 0]
df_embedding_no_out = df_embedding_no_out.drop(["outliers"], axis = 1)


df_embedding_with_out = df_embedding.copy()
df_embedding_with_out = df_embedding_with_out.drop(["outliers"], axis = 1)

In [ ]:
df_embedding_no_out

In [ ]:
# Instantiate the clustering model and visualizer
km = KMeans(init="k-means++", random_state=0, n_init="auto")
visualizer = KElbowVisualizer(km, k=(2,15), locate_elbow=True)
 
visualizer.fit(df_embedding_with_out)        # Fit the data to the visualizer
visualizer.show()  

In [ ]:
n_k = visualizer.elbow_value_

In [ ]:
clusters = KMeans(n_clusters=n_k, init = "k-means++").fit(df_embedding_no_out)
clusters_predict = clusters.predict(df_embedding_no_out)


In [ ]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained(
    model_path_or_repo_id="/mnt/DATA/THESIS/playground/models/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
    model_file="./models/mistral-7b-instruct-v0.1.Q5_K_M.gguf",
    model_type="mistral", 
    gpu_layers=20)

In [ ]:
from nltk.corpus import stopwords
import nltk
# Download the stopwords from NLTK
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

In [ ]:
# for i in zip(sentences,clusters_predict):
#     print(i)

from collections import Counter

data = pd.DataFrame(columns=["sentences", "topic"], data=zip(sentences, clusters_predict))
topic_bow = {}

for i in data['topic'].unique():
    # Initialize a Counter for each topic
    topic_bow[i] = Counter()
    for j in data[data.topic == i].sentences:
        # Tokenize the sentence
        words = j.split()
        # Remove stop words and punctuation
        words = [word for word in words if word not in stop_words and word.isalpha() and not word.startswith("htt")]
        # Add the words to the topic bag of words
        topic_bow[i].update(words)


    # Get the 10 most common words in the topic
    topic_bow[i] = Counter(topic_bow[i]).most_common(20)



In [ ]:

for idx,i in topic_bow.items():
    text = ", ".join(j[0] for j in i)
    print(text)

    topic = llm(f"""
                    You will be given some words and you should create a topic name to match those words. only write the topic name 
                        Example 1:
                            text: PC, laptop, monitor, AI, internet.
                            topic: tech.
                        
                        Example 2:
                            text: hospital, aspirin, blood, injection, doctor
                            topic: Medicine.
                        
                        Generate in single word: 
                            text: {text},
                            topic:
                        """, temperature=0)
    print(">", topic)

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score

"""
The Davies Bouldin index is defined as the average similarity measure of each cluster with its most similar cluster, where similarity is the ratio of within-cluster distances to between-cluster distances.
The minimum value of the DB Index is 0, whereas a smaller value (closer to 0) represents a better model that produces better clusters.
"""
print(f"Davies bouldin score: {davies_bouldin_score(df_embedding_no_out,clusters_predict)}")

"""
Calinski Harabaz Index -> Variance Ratio Criterion.
Calinski Harabaz Index is defined as the ratio of the sum of between-cluster dispersion and of within-cluster dispersion.
The higher the index the more separable the clusters.
"""
print(f"Calinski Score: {calinski_harabasz_score(df_embedding_no_out,clusters_predict)}")


"""
The silhouette score is a metric used to calculate the goodness of fit of a clustering algorithm, but can also be used as a method for determining an optimal value of k (see here for more).
Its value ranges from -1 to 1.
A value of 0 indicates clusters are overlapping and either the data or the value of k is incorrect.
1 is the ideal value and indicates that clusters are very dense and nicely separated.
"""
print(f"Silhouette Score: {silhouette_score(df_embedding_no_out,clusters_predict)}")

In [ ]:
def get_pca_2d(df, predict):

    pca_2d_object = prince.PCA(
    n_components=2,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
    )

    pca_2d_object.fit(df)

    df_pca_2d = pca_2d_object.transform(df)
    df_pca_2d.columns = ["comp1", "comp2"]
    df_pca_2d["cluster"] = predict

    return pca_2d_object, df_pca_2d



def get_pca_3d(df, predict):

    pca_3d_object = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
    )

    pca_3d_object.fit(df)

    df_pca_3d = pca_3d_object.transform(df)
    df_pca_3d.columns = ["comp1", "comp2", "comp3"]
    df_pca_3d["cluster"] = predict

    return pca_3d_object, df_pca_3d



def plot_pca_3d(df, title = "PCA Space", opacity=0.8, width_line = 0.1):

    df = df.astype({"cluster": "object"})
    df = df.sort_values("cluster")

    fig = px.scatter_3d(df, 
                        x='comp1', 
                        y='comp2', 
                        z='comp3',
                        color='cluster',
                        template="plotly",
                        
                        # symbol = "cluster",
                        
                        color_discrete_sequence=px.colors.qualitative.Vivid,
                        title=title).update_traces(
                            # mode = 'markers',
                            marker={
                                "size": 4,
                                "opacity": opacity,
                                # "symbol" : "diamond",
                                "line": {
                                    "width": width_line,
                                    "color": "black",
                                }
                            }
                        ).update_layout(
                                width = 1000, 
                                height = 800, 
                                autosize = False, 
                                showlegend = True,
                                legend=dict(title_font_family="Times New Roman",
                                            font=dict(size= 20)),
                                scene = dict(xaxis=dict(title = 'comp1', titlefont_color = 'black'),
                                            yaxis=dict(title = 'comp2', titlefont_color = 'black'),
                                            zaxis=dict(title = 'comp3', titlefont_color = 'black')),
                                font = dict(family = "Gilroy", color  = 'black', size = 15))
                      
    
    fig.show()


def plot_pca_2d(df, title = "PCA Space", opacity=0.8, width_line = 0.1):

    df = df.astype({"cluster": "object"})
    df = df.sort_values("cluster")

    fig = px.scatter(df, 
                        x='comp1', 
                        y='comp2', 
                        color='cluster',
                        template="plotly",
                        # symbol = "cluster",
                        
                        color_discrete_sequence=px.colors.qualitative.Vivid,
                        title=title).update_traces(
                            # mode = 'markers',
                            marker={
                                "size": 8,
                                "opacity": opacity,
                                # "symbol" : "diamond",
                                "line": {
                                    "width": width_line,
                                    "color": "black",
                                }
                            }
                        ).update_layout(
                                width = 800, 
                                height = 700, 
                                autosize = False, 
                                showlegend = True,
                                legend=dict(title_font_family="Times New Roman",
                                            font=dict(size= 20)),
                                scene = dict(xaxis=dict(title = 'comp1', titlefont_color = 'black'),
                                            yaxis=dict(title = 'comp2', titlefont_color = 'black'),
                                            ),
                                font = dict(family = "Gilroy", color  = 'black', size = 15))
                        
        
    fig.show()

In [ ]:
pca_3d_object, df_pca_3d = get_pca_3d(df_embedding_no_out, clusters_predict)
plot_pca_3d(df_pca_3d, title = "PCA Space", opacity=1, width_line = 0.1)
print("The variability is :", pca_3d_object.eigenvalues_summary)

In [ ]:
pca_2d_object, df_pca_2d = get_pca_2d(df_embedding_no_out, clusters_predict)
plot_pca_2d(df_pca_2d, title = "PCA Space", opacity=1, width_line = 0.2)

In [ ]:
sampling_data = df_embedding_no_out.sample(frac=0.5, replace=True, random_state=1)
sampling_clusters = pd.DataFrame(clusters_predict).sample(frac=0.5, replace=True, random_state=1)[0].values

df_tsne_3d = TSNE(
                  n_components=3, 
                  learning_rate=500, 
                  init='random', 
                  perplexity=200, 
                  n_iter = 5000).fit_transform(sampling_data)

df_tsne_3d = pd.DataFrame(df_tsne_3d, columns=["comp1", "comp2",'comp3'])
df_tsne_3d["cluster"] = sampling_clusters
plot_pca_3d(df_tsne_3d, title = "T-SNE Space", opacity=1, width_line = 0.1)

In [ ]:
plot_pca_3d(df_tsne_3d, title = "T-SNE Space", opacity=0.1, width_line = 0.1)

In [ ]:
df_tsne_2d = TSNE(
                  n_components=2, 
                  learning_rate=500, 
                  init='random', 
                  perplexity=200, 
                  n_iter = 5000).fit_transform(sampling_data)

df_tsne_2d = pd.DataFrame(df_tsne_2d, columns=["comp1", "comp2"])
df_tsne_2d["cluster"] = sampling_clusters

plot_pca_2d(df_tsne_2d, title = "PCA Space", opacity=0.5, width_line = 0.5)